In [2]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [3]:
# Hyper Parameters
EPOCH = 1
BATCH_SIZE = 64
TIME_STEP = 28        # rnn time step / image height
INPUT_SIZE = 28       # rnn input size / image width
LR = 0.001
DOWNLOAD_MNIST = False

In [4]:
train_data = dsets.MNIST(
    root='./mnist', 
    train=True, 
    transform=transforms.ToTensor(),
    download=DOWNLOAD_MNIST,
)

In [6]:
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
# load test data
test_data = dsets.MNIST(
    root='./mnist',
    train=False,
)
test_x = torch.unsqueeze(test_data.data, dim=1).type(torch.FloatTensor)[:2000]/255
test_y = test_data.targets[:2000]

In [9]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn = nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=64,
            num_layers=1,
            batch_first=True,   # True: (batch, time_step, input)
                                # False: (time_step, batch, input)
        )
        self.out = nn.Linear(64, 10)
        
    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)   # x: (batch, time_step, input)
        output = self.out(r_out[:, -1, : ])
        return output
    
rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [11]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

In [13]:
# training and testing
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):
        b_x = b_x.view(-1, 28, 28)            # reshape x to (batch, time_step, input)
        
        output = rnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            test_output = rnn(test_x.view(-1, 28, 28))
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, ' | Step: ', step, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)


Epoch:  0  | Step:  0 | train loss: 2.3100 | test accuracy: 0.09
Epoch:  0  | Step:  50 | train loss: 2.0892 | test accuracy: 0.31
Epoch:  0  | Step:  100 | train loss: 1.3039 | test accuracy: 0.56
Epoch:  0  | Step:  150 | train loss: 0.7972 | test accuracy: 0.70
Epoch:  0  | Step:  200 | train loss: 0.6543 | test accuracy: 0.73
Epoch:  0  | Step:  250 | train loss: 0.5907 | test accuracy: 0.77
Epoch:  0  | Step:  300 | train loss: 0.5729 | test accuracy: 0.81
Epoch:  0  | Step:  350 | train loss: 0.3639 | test accuracy: 0.84
Epoch:  0  | Step:  400 | train loss: 0.6499 | test accuracy: 0.84
Epoch:  0  | Step:  450 | train loss: 0.3576 | test accuracy: 0.86
Epoch:  0  | Step:  500 | train loss: 0.4404 | test accuracy: 0.87
Epoch:  0  | Step:  550 | train loss: 0.5649 | test accuracy: 0.88
Epoch:  0  | Step:  600 | train loss: 0.2122 | test accuracy: 0.89
Epoch:  0  | Step:  650 | train loss: 0.4332 | test accuracy: 0.89
Epoch:  0  | Step:  700 | train loss: 0.4372 | test accuracy: 0.9

In [15]:
test_output = rnn(test_x.view(-1, 28, 28)[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, ' prediction number')
print(test_y[:10].numpy(), ' real number')

[7 2 1 0 4 1 4 9 6 4]  prediction number
[7 2 1 0 4 1 4 9 5 9]  real number
